# 🌐 Base de Données de l'Étude


In [6]:
# Création de la liste de triplets (région, longitude, latitude)
region_centroides = [('Guadeloupe', -61.53983304107784, 16.197645826552904),
 ('Guyane', -53.23993971851178, 3.915166862499844),
 ('Martinique', -61.019378740773654, 14.654870858984358),
 ('Mayotte', 45.14743542382146, -12.820616875536656),
 ('Auvergne-Rhône-Alpes', 4.536118586333609, 45.51464202656834),
 ('Occitanie', 2.143922833937654, 43.70250024549978),
 ('Bretagne', -2.841462676465886, 48.17754922086023),
 ('Nouvelle-Aquitaine', 0.21271860548694332, 45.2060395396088),
 ('Normandie', 0.11093239896467046, 49.12006667987373),
 ("Provence-Alpes-Côte d'Azur", 6.061004522552231, 43.95878857291765),
 ('Pays de la Loire', -0.81678598261894, 47.47810354487287),
 ('Grand Est', 5.613112231600144, 48.68897740160863),
 ('Bourgogne-Franche-Comté', 4.806856915301157, 47.2342819451994),
 ('Île-de-France', 2.503401392552759, 48.70931550109074),
 ('Centre-Val de Loire', 1.684391322381998, 47.48477508284454),
 ('Hauts-de-France', 2.7713028658475474, 49.969593961262426),
 ('Corse', 9.106151755156151, 42.15151137562012),
 ('La Réunion', 55.53250880056539, -21.133202961171154)]

Ce script collecte des **données horaires de qualité de l'air** via l'API Open-Meteo pour différentes régions. 
Il récupère les polluants clés (**PM10**, **PM2.5**, **CO**, **NO₂**, **SO₂**, **Ozone**)  et quelques données sur le climat
et les organise dans un tableau structuré.

---

## **Fonctionnalités**
- Utilisation d'un **cache** et de **répétitions automatiques** pour des requêtes fiables.
- Organisation des données dans des **DataFrames pandas**.
- Ajout d'une colonne pour **extraire les dates** sans les heures.
- Fusion des données pour une **analyse centralisée**.





### Récupération des données sur les polluants 

In [2]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from openmeteo_requests import Client


# Création d'une session avec cache
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = Client(session=retry_session)

# Création d'un dictionnaire pour stocker les DataFrames
region_dataframes = {}

# Boucle pour récupérer les données pour chaque région
for region, longitude, latitude in region_centroides:
    try:
        url = "https://air-quality-api.open-meteo.com/v1/air-quality"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current": ["european_aqi", "us_aqi"],
            "hourly": ["pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone"],
            "start_date": "2023-01-01",
            "end_date": "2024-11-30"
        }

        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]  # Première réponse, si plusieurs localisations

        # Récupération des données horaires
        hourly = response.Hourly()
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "region": [region] * len(pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )),
            "pm10": hourly.Variables(0).ValuesAsNumpy(),
            "pm2_5": hourly.Variables(1).ValuesAsNumpy(),
            "carbon_monoxide": hourly.Variables(2).ValuesAsNumpy(),
            "carbon_dioxide": hourly.Variables(3).ValuesAsNumpy(),
            "nitrogen_dioxide": hourly.Variables(4).ValuesAsNumpy(),
            "sulphur_dioxide": hourly.Variables(5).ValuesAsNumpy(),
            "ozone": hourly.Variables(6).ValuesAsNumpy()
        }

        # Stockage du DataFrame dans le dictionnaire
        region_dataframes[region] = pd.DataFrame(data=hourly_data)

    except Exception as e:
        print(f"Erreur lors de la récupération des données pour {region}: {e}")

# Concaténation tous les DataFrames
combined_dataframe_air = pd.concat(region_dataframes.values(), ignore_index=True)
combined_dataframe_air['date'] = pd.to_datetime(combined_dataframe_air['date'])
# Création d'une nouvelle colonne 'day' contenant uniquement la date (sans l'heure)
#combined_dataframe_air['day'] = combined_dataframe_air['date'].dt.date
combined_dataframe_air.insert(1,"day",combined_dataframe_air["date"].dt.date)



In [3]:
combined_dataframe_air

,date,day,region,pm10,pm2_5,carbon_monoxide,carbon_dioxide,nitrogen_dioxide,sulphur_dioxide,ozone
0,2023-01-01 00:00:00+00:00,2023-01-01,Guadeloupe,20.500000,11.3,89.0,NaN,3.70,0.30,62.0
1,2023-01-01 01:00:00+00:00,2023-01-01,Guadeloupe,21.700001,11.9,89.0,NaN,3.45,0.30,62.0
2,2023-01-01 02:00:00+00:00,2023-01-01,Guadeloupe,21.799999,12.0,88.0,NaN,3.15,0.25,63.0
3,2023-01-01 03:00:00+00:00,2023-01-01,Guadeloupe,20.700001,11.4,88.0,NaN,2.60,0.25,64.0
4,2023-01-01 04:00:00+00:00,2023-01-01,Guadeloupe,19.400000,10.7,88.0,NaN,2.10,0.25,64.0
...,...,...,...,...,...,...,...,...,...,...
302395,2024-11-30 19:00:00+00:00,2024-11-30,La Réunion,10.000000,7.1,100.0,459.0,3.60,2.50,56.0
302396,2024-11-30 20:00:00+00:00,2024-11-30,La Réunion,9.800000,7.1,95.0,455.0,4.20,2.30,52.0
302397,2024-11-30 21:00:00+00:00,2024-11-30,La Réunion,9.700000,7.0,91.0,453.0,4.50,2.20,49.0
302398,2024-11-30 22:00:00+00:00,2024-11-30,La Réunion,9.500000,6.9,90.0,455.0,4.40,2.10,47.0


### Récupération des données sur le climat comme la température, etc.

In [4]:
# Création d'une session avec cache
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = Client(session=retry_session)

# Création d'un dictionnaire pour stocker les DataFrames
region_dataframes = {}

# Boucle pour récupérer les données pour chaque région
for region, longitude, latitude in region_centroides:
    try:
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "surface_pressure", "wind_speed_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm"],
            "start_date": "2023-01-01",
            "end_date": "2024-11-30"
        }

        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]  # Première réponse, si plusieurs localisations

        # Récupération des données horaires
        hourly = response.Hourly()
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "region": [region] * len(pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )),
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
            "precipitation": hourly.Variables(2).ValuesAsNumpy(),
            "surface_pressure": hourly.Variables(3).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(4).ValuesAsNumpy(),
            "soil_temperature_0_to_7cm": hourly.Variables(5).ValuesAsNumpy(),
            "soil_moisture_0_to_7cm": hourly.Variables(6).ValuesAsNumpy(),
            
        }

        # Stockage du DataFrame dans le dictionnaire
        region_dataframes[region] = pd.DataFrame(data=hourly_data)

    except Exception as e:
        print(f"Erreur lors de la récupération des données pour {region}: {e}")
        
    # Concaténation tous les DataFrames
combined_dataframe_climat = pd.concat(region_dataframes.values(), ignore_index=True)
combined_dataframe_climat = combined_dataframe_climat.drop(columns=["region","date"])

### Concaténation des deux Dataframes

In [5]:
df=pd.concat([combined_dataframe_air,combined_dataframe_climat],axis=1)
df

,date,day,region,pm10,pm2_5,carbon_monoxide,carbon_dioxide,nitrogen_dioxide,sulphur_dioxide,ozone,temperature_2m,relative_humidity_2m,precipitation,surface_pressure,wind_speed_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm
0,2023-01-01 00:00:00+00:00,2023-01-01,Guadeloupe,20.500000,11.3,89.0,NaN,3.70,0.30,62.0,23.770000,84.863304,0.0,1017.400024,9.422101,24.020000,0.501
1,2023-01-01 01:00:00+00:00,2023-01-01,Guadeloupe,21.700001,11.9,89.0,NaN,3.45,0.30,62.0,23.719999,85.643173,0.0,1017.599976,9.885262,23.670000,0.499
2,2023-01-01 02:00:00+00:00,2023-01-01,Guadeloupe,21.799999,12.0,88.0,NaN,3.15,0.25,63.0,23.520000,86.947586,0.0,1017.700012,9.720000,23.420000,0.498
3,2023-01-01 03:00:00+00:00,2023-01-01,Guadeloupe,20.700001,11.4,88.0,NaN,2.60,0.25,64.0,23.270000,88.269188,0.0,1017.400024,7.559999,23.219999,0.497
4,2023-01-01 04:00:00+00:00,2023-01-01,Guadeloupe,19.400000,10.7,88.0,NaN,2.10,0.25,64.0,22.469999,94.082932,0.2,1017.299988,8.825508,23.020000,0.497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302395,2024-11-30 19:00:00+00:00,2024-11-30,La Réunion,10.000000,7.1,100.0,459.0,3.60,2.50,56.0,10.291500,93.833534,0.0,824.523071,3.617955,12.941500,0.235
302396,2024-11-30 20:00:00+00:00,2024-11-30,La Réunion,9.800000,7.1,95.0,455.0,4.20,2.30,52.0,9.891500,93.814590,0.0,823.957458,3.847960,12.441500,0.235
302397,2024-11-30 21:00:00+00:00,2024-11-30,La Réunion,9.700000,7.0,91.0,453.0,4.50,2.20,49.0,9.391500,97.342743,0.0,823.007019,3.065289,11.991500,0.234
302398,2024-11-30 22:00:00+00:00,2024-11-30,La Réunion,9.500000,6.9,90.0,455.0,4.40,2.10,47.0,8.891500,99.663048,0.0,822.137085,2.979530,11.591500,0.234


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302400 entries, 0 to 302399
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype              
---  ------                     --------------   -----              
 0   date                       302400 non-null  datetime64[ns, UTC]
 1   day                        302400 non-null  object             
 2   region                     302400 non-null  object             
 3   pm10                       302400 non-null  float32            
 4   pm2_5                      302400 non-null  float32            
 5   carbon_monoxide            302400 non-null  float32            
 6   carbon_dioxide             15552 non-null   float32            
 7   nitrogen_dioxide           302400 non-null  float32            
 8   sulphur_dioxide            302400 non-null  float32            
 9   ozone                      302400 non-null  float32            
 10  temperature_2m             302400 non-null  float32     

# Commentaires sur la base de données

## 1. Structure cohérente
- La base est bien structurée avec des colonnes précises : date, région, paramètres de qualité de l'air (PM10, PM2.5, etc.), conditions météorologiques et variables environnementales.
- Cette structure facilite l'analyse et le traitement des données.

## 2. Données temporelles détaillées
- Les données sont collectées à une résolution horaire, ce qui permet des analyses fines comme :
  - La variabilité intra-journalière.
  - Les tendances horaires sur plusieurs jours ou mois.

## 3. Données manquantes
- La colonne **carbon_dioxide** contient des valeurs manquantes (`NaN`), nécessitant un traitement :
  - Imputation (moyenne, médiane, interpolation) ou suppression selon l'analyse souhaitée.
- Une vérification est nécessaire pour identifier d'autres colonnes avec des valeurs manquantes.

## 4. Régions multiples
- La base couvre toutes les régions de la France, même les régions d'outre-mer

## 5. Diversité des variables
- Un large éventail de variables est disponible :
  - Polluants atmosphériques (PM10, PM2.5, dioxyde d'azote, etc.).
  - Données météorologiques (température, humidité relative, précipitations, vitesse du vent).
- Ces données permettent des analyses multidimensionnelles.

## 6. Période longue
- La base couvre près de deux ans (janvier 2023 à novembre 2024).
- Cela offre des possibilités d'analyse des tendances saisonnières ou annuelles.